Convert '/mnt/prepared/SupplyChain/StoreOrder/StoreOrderLine' --> '/mnt/thorcf2/clean/supp/dedi/storeorder'

In [49]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import year, month
from pyspark.sql.functions import to_timestamp,date_format

spark = SparkSession.builder.getOrCreate()

In [50]:
df1 = spark.read.load('/mnt/prepared/SupplyChain/StoreOrder/StoreOrderLine')
df1.printSchema()

root
 |-- InsertDate: timestamp (nullable = true)
 |-- StoreNo: integer (nullable = true)
 |-- WarehouseNo: integer (nullable = true)
 |-- DeliveryStreamNo: integer (nullable = true)
 |-- DeliveryDateTime: timestamp (nullable = true)
 |-- StoreOrderNo: integer (nullable = true)
 |-- Soino: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- OrderLineId: integer (nullable = true)
 |-- PromotionInd: string (nullable = true)
 |-- InitialPromotionInd: string (nullable = true)
 |-- SupplierCompanyGln: string (nullable = true)
 |-- SupplierNo: integer (nullable = true)
 |-- AddUnorderedInd: string (nullable = true)
 |-- EmergencyOrderInd: string (nullable = true)
 |-- date: date (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)



In [51]:
df2 = spark.read.load('dbfs:/mnt/thorcf2/clean/supp/dedi/storeorder')
df2.printSchema()

root
 |-- Header: struct (nullable = false)
 |    |-- Event: struct (nullable = false)
 |    |    |-- Action: string (nullable = true)
 |    |    |-- DateTime: timestamp (nullable = true)
 |-- Payload: struct (nullable = false)
 |    |-- StoreOrder: struct (nullable = false)
 |    |    |-- OpCo: struct (nullable = false)
 |    |    |    |-- CountryCode: string (nullable = true)
 |    |    |    |-- GLN: long (nullable = true)
 |    |    |-- StoreNo: integer (nullable = true)
 |    |    |-- WarehouseNo: integer (nullable = true)
 |    |    |-- WarehouseGln: long (nullable = true)
 |    |    |-- StoreOrderNo: integer (nullable = true)
 |    |    |-- MessageCreationDatetime: timestamp (nullable = true)
 |    |    |-- DeliveryStreamNo: integer (nullable = true)
 |    |    |-- DeliveryDateTime: timestamp (nullable = true)
 |    |    |-- DelivererNo: integer (nullable = true)
 |    |    |-- SupplyMethod: string (nullable = true)
 |    |    |-- StoreOrderLine: array (nullable = true)
 |    |  

In [52]:
df2 = df2.select("Header.*", "Payload.*", "DeliveryDatePartitionKey", "__meta.*")
df2 = df2.select("Event.*", "StoreOrder.*", "DeliveryDatePartitionKey", "ModifiedTimestamp")
df2 = df2.select("Action", "DateTime", "Opco.*", "StoreNo", "WarehouseNo","StoreOrderNo","MessageCreationDatetime","DeliveryStreamNo","DeliveryDateTime","DelivererNo","SupplyMethod","StoreOrderLine","DeliveryDatePartitionKey","ModifiedTimestamp")
df2.printSchema()

root
 |-- Action: string (nullable = true)
 |-- DateTime: timestamp (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- GLN: long (nullable = true)
 |-- StoreNo: integer (nullable = true)
 |-- WarehouseNo: integer (nullable = true)
 |-- StoreOrderNo: integer (nullable = true)
 |-- MessageCreationDatetime: timestamp (nullable = true)
 |-- DeliveryStreamNo: integer (nullable = true)
 |-- DeliveryDateTime: timestamp (nullable = true)
 |-- DelivererNo: integer (nullable = true)
 |-- SupplyMethod: string (nullable = true)
 |-- StoreOrderLine: array (nullable = true)
 |    |-- element: struct (containsNull = false)
 |    |    |-- SoiNo: long (nullable = true)
 |    |    |-- Quantity: integer (nullable = true)
 |    |    |-- OrderLineId: integer (nullable = true)
 |    |    |-- PromotionInd: string (nullable = true)
 |    |    |-- InitialPromotionInd: string (nullable = true)
 |    |    |-- SupplierCompanyGln: long (nullable = true)
 |    |    |-- SupplierNo: integer (nullable =

In [53]:
df2 = df2.select("Action","DateTime","CountryCode","GLN","StoreNo","WarehouseNo","StoreOrderNo","MessageCreationDatetime",
                 "DeliveryStreamNo","DeliveryDateTime","DelivererNo","SupplyMethod",*[F.col('StoreOrderLine')[i].alias(f'StoreOrderLine{i}') for i in range(1)],
                 "DeliveryDatePartitionKey","ModifiedTimestamp")


df2 = df2.select("Action","DateTime","CountryCode","GLN","StoreNo","WarehouseNo","StoreOrderNo","MessageCreationDatetime",
                 "DeliveryStreamNo","DeliveryDateTime","DelivererNo","SupplyMethod","StoreOrderLine0.*","DeliveryDatePartitionKey","ModifiedTimestamp")
df2.printSchema()

root
 |-- Action: string (nullable = true)
 |-- DateTime: timestamp (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- GLN: long (nullable = true)
 |-- StoreNo: integer (nullable = true)
 |-- WarehouseNo: integer (nullable = true)
 |-- StoreOrderNo: integer (nullable = true)
 |-- MessageCreationDatetime: timestamp (nullable = true)
 |-- DeliveryStreamNo: integer (nullable = true)
 |-- DeliveryDateTime: timestamp (nullable = true)
 |-- DelivererNo: integer (nullable = true)
 |-- SupplyMethod: string (nullable = true)
 |-- SoiNo: long (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- OrderLineId: integer (nullable = true)
 |-- PromotionInd: string (nullable = true)
 |-- InitialPromotionInd: string (nullable = true)
 |-- SupplierCompanyGln: long (nullable = true)
 |-- SupplierNo: integer (nullable = true)
 |-- AddUnorderedInd: string (nullable = true)
 |-- EmergencyOrderInd: string (nullable = true)
 |-- DeliveryDatePartitionKey: integer (nullable = true)
 |--

DF1 : DF2
- InsertDate : DateTime
- StoreNo : StoreNo
- WarehouseNo : WarehouseNo
- DeliveryStreamNo : DeliveryStreamNo
- DeliveryDateTime : DeliveryDateTime
- StoreOrderNo : StoreOrderNo
- Soino : SoiNo -> ALIAS?
- Quantity : Quantity
- OrderLineId : OrderLineId
- PromotionInd : PromotionInd
- InitialPromotionInd : InitialPromotionInd
- SupplierCompanyGln : SupplierCompanyGln
- SupplierNo : SupplierNo
- AddUnorderedInd : AddUnorderedInd
- EmergencyOrderInd : EmergencyOrderInd
- date : <font color='red'>AANMAKEN</font>
- year : <font color='red'>AANMAKEN</font>
- month : <font color='red'>AANMAKEN</font>
- day : <font color='red'>AANMAKEN</font>

In [54]:
# Voor date kolom: DeliveryDateTime gebruikt, is dat goed? 
df2 = df2.withColumn("date", to_date(col("DeliveryDateTime")))
df2 = df2.withColumn('year', year(df2.date))
df2 = df2.withColumn('month', month(df2.date))
df2 = df2.withColumn("date",to_timestamp(col("date"))).withColumn("day", date_format(col("date"), "d"))

In [55]:
df2.printSchema()

root
 |-- Action: string (nullable = true)
 |-- DateTime: timestamp (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- GLN: long (nullable = true)
 |-- StoreNo: integer (nullable = true)
 |-- WarehouseNo: integer (nullable = true)
 |-- StoreOrderNo: integer (nullable = true)
 |-- MessageCreationDatetime: timestamp (nullable = true)
 |-- DeliveryStreamNo: integer (nullable = true)
 |-- DeliveryDateTime: timestamp (nullable = true)
 |-- DelivererNo: integer (nullable = true)
 |-- SupplyMethod: string (nullable = true)
 |-- SoiNo: long (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- OrderLineId: integer (nullable = true)
 |-- PromotionInd: string (nullable = true)
 |-- InitialPromotionInd: string (nullable = true)
 |-- SupplierCompanyGln: long (nullable = true)
 |-- SupplierNo: integer (nullable = true)
 |-- AddUnorderedInd: string (nullable = true)
 |-- EmergencyOrderInd: string (nullable = true)
 |-- DeliveryDatePartitionKey: integer (nullable = true)
 |--

In [8]:
df2.show()

+------+-------------------+-----------+-------------+-------+-----------+------------+-----------------------+----------------+-------------------+-----------+------------+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+------------------------+--------------------+-------------------+----+-----+---+
|Action|           DateTime|CountryCode|          GLN|StoreNo|WarehouseNo|StoreOrderNo|MessageCreationDatetime|DeliveryStreamNo|   DeliveryDateTime|DelivererNo|SupplyMethod| SoiNo|Quantity|OrderLineId|PromotionInd|InitialPromotionInd|SupplierCompanyGln|SupplierNo|AddUnorderedInd|EmergencyOrderInd|DeliveryDatePartitionKey|   ModifiedTimestamp|               date|year|month|day|
+------+-------------------+-----------+-------------+-------+-----------+------------+-----------------------+----------------+-------------------+-----------+------------+------+--------+-----------+------------+-------------------+------

In [17]:
# Controleren op overeenkomsten
df2.select('year').distinct().show()
df1.select('year').distinct().sort('year').show()

+----+
|year|
+----+
|2023|
|2022|
+----+

+----+
|year|
+----+
|2015|
|2016|
|2017|
|2018|
|2019|
|2020|
|2021|
|2022|
|2023|
+----+

